following https://github.com/gkamradt/langchain-tutorials/blob/main/agents/Agents.ipynb

In [5]:
# load .env
import os
from dotenv import load_dotenv

load_dotenv()

# check HF token and SERPAPI key have been loaded
assert "OPENAI_API_KEY" in os.environ
assert "SERPAPI_API_KEY" in os.environ
assert "WOLFRAM_ALPHA_APPID" in os.environ

In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

## Tools

### Serp API

In [3]:
# get llm
llm = OpenAI(temperature=0)

# init tools
tool_names = ["serpapi"]
tools = load_tools(tool_names)

# init agent 
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [4]:
agent.run("Who will be in the 2023 Champions League semi-finals?")



> Entering new AgentExecutor chain...
 I need to find out who is currently in the Champions League
Action: Search
Action Input: "2023 Champions League semi-finals"
Observation: The semifinals are set to start beginning on Tuesday, May 9. MORE: Champions League prize money breakdown 2022/2023. Semifinals. The first team hosts the first ...
Thought: I need to find out which teams are in the Champions League
Action: Search
Action Input: "2022/2023 Champions League teams"
Observation: No good search result found
Thought: I need to find out which teams qualified for the Champions League
Action: Search
Action Input: "2022/2023 Champions League qualifiers"
Observation: Qualifying: 21 June – 24 August 2022. Competition proper: 6 September 2022 – 10 June 2023 · Competition proper: 32. Total: 78 (from 53 associations) · 120.
Thought: I need to find out which teams qualified for the Champions League semi-finals
Action: Search
Action Input: "2022/2023 Champions League semi-finals qualifiers"
Obs

'Real Madrid and AC Milan have qualified for the 2023 Champions League semi-finals.'

that's pretty good. That's half the answer (the 2 teams that qualifies the day before yesterday), and the search results were not terribly helpful. M.City and Inter Milan also advanced yesterday.

### Wolfram Alpha

In [7]:
llm = OpenAI(temperature=0)

tool_names = ["wolfram-alpha", "serpapi"]
tools = load_tools(tool_names)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [8]:
agent.run("What is the area of the curve the Earth describes when orbiting around the Sun?")



> Entering new AgentExecutor chain...
 I need to find the area of a curved path
Action: Wolfram Alpha
Action Input: area of curved path
Observation: Wolfram Alpha wasn't able to answer it
Thought: I need to find the circumference of the Earth's orbit
Action: Wolfram Alpha
Action Input: circumference of Earth's orbit
Observation: Assumption: Earth | orbit circumference 
Answer: 9.399×10^8 km (kilometers)
Thought: I need to calculate the area of a circle with this circumference
Action: Wolfram Alpha
Action Input: area of circle with circumference 9.399×10^8 km
Observation: Assumption: circle | circumference 9.399×10^8 km (kilometers) | area enclosed 
Answer: 7.03×10^16 km^2 (square kilometers)
Thought: I now know the final answer
Final Answer: The area of the curve the Earth describes when orbiting around the Sun is 7.03×10^16 km^2 (square kilometers).

> Finished chain.


'The area of the curve the Earth describes when orbiting around the Sun is 7.03×10^16 km^2 (square kilometers).'

that's an interesting way of getting the result. The order of magnitude looks correct. But I wanted it to use the elliptical orbit. Lets try again:

In [10]:
agent.run("Taking into account the eccentricity, what is the area of Earth's orbit around the Sun?")



> Entering new AgentExecutor chain...
 I need to find the area of an ellipse given its eccentricity.
Action: Wolfram Alpha
Action Input: Area of an ellipse given its eccentricity
Observation: Wolfram Alpha wasn't able to answer it
Thought: I need to find the area of an ellipse given its major and minor axes
Action: Wolfram Alpha
Action Input: Area of an ellipse given its major and minor axes
Observation: Wolfram Alpha wasn't able to answer it
Thought: I need to find the major and minor axes of an ellipse given its eccentricity
Action: Wolfram Alpha
Action Input: Major and minor axes of an ellipse given its eccentricity
Observation: Wolfram Alpha wasn't able to answer it
Thought: I need to find the eccentricity of Earth's orbit around the Sun
Action: Search
Action Input: Eccentricity of Earth's orbit around the Sun
Observation: about 0.0167
Thought: I need to find the area of an ellipse given its major and minor axes
Action: Wolfram Alpha
Action Input: Area of an ellipse given its maj

KeyboardInterrupt: 

I interrupted it before it burns through my free API calls.

Lets try it once more dividing the question in 2.

In [12]:
agent.run("What is the shape of Earth's orbit? What is the area of Earth's orbit?")



> Entering new AgentExecutor chain...
 I need to find out the shape and area of Earth's orbit.
Action: Wolfram Alpha
Action Input: Shape and area of Earth's orbit
Observation: Assumption: Earth | shape | Earth | surface area | Earth | orbital properties 
Answer: shape | oblate spheroid (ignoring surface features)
surface area | 5.1007×10^8 km^2 (square kilometers)
Thought: I now know the final answer
Final Answer: The shape of Earth's orbit is an oblate spheroid and the area is 5.1007×10^8 km^2 (square kilometers).

> Finished chain.


"The shape of Earth's orbit is an oblate spheroid and the area is 5.1007×10^8 km^2 (square kilometers)."

This looks like it's off by 8 orders of magnitude.